In [1]:
# MSM 107 analysis
# ROSINA data
# plotting transection

import pandas as pd
import numpy as np
import os, glob, statistics, math, gsw
from tkinter import filedialog
import matplotlib.pyplot as plt
import scipy.interpolate
import matplotlib.tri as tri

In [2]:

particle_path = os.path.abspath("/Volumes/SEGATE1T/MSM107/03_ROSINA/_Results/08_ResultsPython/08_ResultsPython")

tran1=["Rosi03/IdroCTD-02/BinnedCTD", "Rosi04/IdroCTD-02/BinnedCTD",
"Rosi05/IdroCTD-02/BinnedCTD", "Rosi06/IdroCTD-02/BinnedCTD",
"Rosi07/IdroCTD-02_Img148Removed/BinnedCTD"]
tran2=["Rosi18/IdroCTD-02/BinnedCTD","Rosi10/IdroCTD-02/BinnedCTD", "Rosi11/IdroCTD-02/BinnedCTD",
"Rosi12/IdroCTD-02/BinnedCTD"]
tran3=["Rosi15/IdroCTD-02/BinnedCTD","Rosi16/IdroCTD-02/BinnedCTD", "Rosi17/IdroCTD-02/BinnedCTD",
"Rosi14/IdroCTD-02/BinnedCTD"]

vtran1=["Rosi05/IdroCTD-02/BinnedCTD", "Rosi12/IdroCTD-02/BinnedCTD",
"Rosi16/IdroCTD-02/BinnedCTD"]
vtran2=["Rosi07/IdroCTD-02_Img148Removed/BinnedCTD", "Rosi14/IdroCTD-02/BinnedCTD"]



#"Rosi02/ManDepth-Img584u577removed/BinnedCTD"
CTD_path = os.path.abspath("/Volumes/SEGATE1T/MSM107/04_CTD-Idronaut")
CTDtran1=["02-Profile03/MSM107-P003-C01.txt",
"03-Profile04/MSM107-P004-C01.txt", "04-Profile05/MSM107-P005-C01.txt",
"05-Profile06/MSM107-P006-C01.txt", "06-Profile07/MSM107-P007-C03.txt"]
CTDtran2=["16-Profile18/MSM107-P018-C01.txt", "08-Profile10/MSM107-P010-C01.txt",
"09-Profile11/MSM107-P011-C01.txt", "10-Profile12/MSM107-P012-C03.txt"]
CTDtran3=["13-Profile15/MSM107-P015-C01.txt",
"14-Profile16/MSM107-P016-C01.txt", "15-Profile17/MSM107-P017-C02.txt", "12-Profile14/MSM107-P014-C01.txt"]


CTDvtran1=["04-Profile05/MSM107-P005-C01.txt", "10-Profile12/MSM107-P012-C03.txt",
"14-Profile16/MSM107-P016-C01.txt"]
CTDvtran2=["06-Profile07/MSM107-P007-C03.txt", "12-Profile14/MSM107-P014-C01.txt"]

plot_path = os.path.abspath("/Volumes/SEGATE1T/MSM107/plot")

In [3]:
# some definitions
# binned definition
def binned (values, depths, bin):
    df = pd.DataFrame(list(zip(values, depths)), columns=["value", "depth"])
    max_d = max(depths)
    nBins = int(max_d / bin) + 1 # 5 m bins
    bin_value = []
    bin_depth = []
    for n in range(0, nBins):
        each_df = df.loc[(df["depth"] > n*bin) & (df["depth"] < (n+1)*bin) ]
        if each_df.empty == True:
            continue
        else:
            value_mean = each_df["value"].mean()
            bin_value.append(value_mean) 
            bin_depth.append(n*5)
    return bin_value, bin_depth

def tocontourf (x_list, y_list, z_list):
    minx, maxx, miny, maxy = float(min(x_list)), float(max(x_list)), float(min(y_list)), float(max(y_list))
    xs, ys = np.linspace(minx, maxx), np.linspace(miny, maxy)
    xgrid, ygrid = np.meshgrid(xs, ys)

    # interpolate
    interp = scipy.interpolate.LinearNDInterpolator(list(zip(x_list, y_list)), z_list)
    zi = interp(xgrid, ygrid)

    return xgrid, ygrid, zi


In [19]:
# plot transection
oxy_list = []
flu_list = []
sal_list = []
temp_list = []
ctd_depth_list = []
ctd_pos_list = []

num_list = []
vol_list = []
depth_list = []
pos_list = []

for k in range(0, len(tran3)):
    p = tran3[k] # particle file
    c = CTDtran3[k] # ctd file

    # reading in data
    #################
    # reading in particle data

    # reading in data
    ##########################
    # reading in particle data
    filePath = os.path.expanduser( os.path.join(particle_path, p) )
    pathParticlesBinned = filePath + "/ParticlesBinned.npy"
    pathVolumeBinned = filePath + "/VolumeBinned.npy"
    #pathAreaBinned = filePath + "/AreaBinned.npy"
    pathDepth = filePath + "/Depth.npy"
    pathConfigFile = filePath +"/ConfigFileHeader.txt"

    # create header and list for dataframe
    particlesBinnedList = []          # list for input data numpy files
    volumeBinnedList = []             # list for input data numpy files
    #areaBinnedList = []               # list for input data numpy files
    depthList = []                    # list for input data numpy files
    eSDList = []                      # list for input data numpy files

    # bins value in mm
    Bins = [0.04875, 0.061425, 0.0773955, 0.0975183, 0.122873, 0.15482, 0.195073, 0.245792, 0.309698, 
        0.39022, 0.491677, 0.619513, 0.780587, 0.983539, 1.23926, 1.56147, 1.96745, 2.47898, 3.12352, 3.93564]
    particlesBinned = np.load(pathParticlesBinned)
    volumeBinned = np.load(pathVolumeBinned)
    #areaBinned = np.load(pathAreaBinned)
    depth = np.load(pathDepth)
    #    eSD = np.load(pathESD)

    configDataFile = open(pathConfigFile, "rt")
    configDataList = configDataFile.read()
    configDataList = configDataList.split("\n")
    configDataFile.close()

    # data converting to dataframe
    numberdf = pd.DataFrame(particlesBinned, columns = Bins)
    volumedf =  pd.DataFrame(volumeBinned, columns = Bins)
    #areadf = pd.DataFrame(areaBinned, columns = Bins)

    # check particle size spectra vs ESD
    sum = numberdf.sum(axis = 0).to_frame()
    sumdf = pd.DataFrame(sum.T, columns = Bins)
    size_interval = [(math.floor(i * 10000)) / 10000.0 for i in Bins] # round down the bins
    size_spectra_l =[]
    esd_l = []
    for i in range(0, len(size_interval)):
        if i == len(size_interval)-1:
            break
        else:
            size_s, size_l = size_interval[i], size_interval[i+1]
            esd_l.append(statistics.mean([size_s, size_l]))
            size_sum = sumdf.loc[:, sumdf.columns[(sumdf.columns > size_s) & (sumdf.columns < size_l)]].sum(axis = 1)
            size_spectra_l.append(size_sum.sum(axis=0)/(size_l-size_s))


    # create vertical profile of particle
    numsum = numberdf.sum(axis = 1).tolist()
    volsum = volumedf.sum(axis = 1).tolist()

    #areasum = areadf.sum(axis = 1).tolist()
    meansum = []
    mediansum = []

    # calculate mean and median for each bins
    for index, row in numberdf.iterrows():
        if statistics.mean(row.values) == 0:
            meansum.append(np.nan)
            mediansum.append(np.nan)
        else:

            mean = statistics.mean( np.repeat(Bins, [int(i) for i in row.values]) )
            median = statistics.median( np.repeat(Bins, [int(i) for i in row.values]) )
            meansum.append(mean)
            mediansum.append(median)

    ###########################
    # readidng in CTD data
    CTDPath = os.path.expanduser( os.path.join(CTD_path, c) )
    CTDdf = pd.read_csv(CTDPath, sep="\t", skiprows=16, header=0, engine="python")

    tempbinned, ctddepthbinned = binned (CTDdf["Temperature"], CTDdf["Depth"], 5) # teperature
    salbinned, ctddepthbinned = binned (CTDdf["Salinity"], CTDdf["Depth"], 5) # salinity
    o2binned, ctddepthbinned = binned (CTDdf["Optical O2%"], CTDdf["Depth"], 5) # oxygen
    flubinned, ctddepthbinned = binned (CTDdf["Fluorometer AutoScale"], CTDdf["Depth"], 5) # fluoresence

    ctdfile = open(CTDPath)
    content = ctdfile.readlines()
    lat = float(content[4].split("\n")[0])
    lon = float(content[3].split("\n")[0])

    
    temp_list.extend(tempbinned)
    sal_list.extend(salbinned)
    oxy_list.extend(o2binned)
    flu_list.extend(flubinned)
    ctd_depth_list.extend(ctddepthbinned)
    ctd_lat_list = [lat]*len(ctddepthbinned)
    ctd_lon_list = [lon]*len(ctddepthbinned)
    ctd_pos_list.extend(ctd_lon_list) ######### change lat and lon

    ###########################
    # binning
    numbinned, depthbinned = binned (numsum, depth, 5)
    volbinned, depthbinned = binned (volsum, depth, 5)
    lat_list = [lat]*len(depthbinned)
    lon_list = [lon]*len(depthbinned)

    ###########################
    # appending
    num_list.extend(numbinned)
    vol_list.extend(volbinned)
    depth_list.extend(depthbinned)
    pos_list.extend(lon_list) ######### change lat and lon

###################################
# 1d lists to grid data for contour
aggrX, aggrY, aggrnum = tocontourf (pos_list, depth_list, num_list)
aggrX, aggrY, aggrvol = tocontourf (pos_list, depth_list, vol_list)

ctdX, ctdY, tempZ = tocontourf (ctd_pos_list, ctd_depth_list, temp_list)
ctdX, ctdY, salZ = tocontourf (ctd_pos_list, ctd_depth_list, sal_list)
ctdX, ctdY, oxyZ = tocontourf (ctd_pos_list, ctd_depth_list, oxy_list)
ctdX, ctdY, fluZ = tocontourf (ctd_pos_list, ctd_depth_list, flu_list)

############################
# figure
fig, ax = plt.subplots(nrows = 3, ncols= 2, figsize=(12, 12), facecolor="white")

ax1 = ax[0,0]
im1 = ax1.contourf(aggrX, aggrY, aggrnum, 50, cmap="rainbow")
fig.colorbar(im1, orientation='vertical', ax=ax1)
ax1.scatter(pos_list, depth_list, s=1.5, color="gray", alpha=0.7)
ax1.invert_yaxis()
ax1.set_title("particle number")

ax2 = ax[0,1]
im2 = ax2.contourf(aggrX, aggrY, aggrvol, 50, cmap="rainbow")
fig.colorbar(im2, orientation='vertical', ax=ax2)
ax2.scatter(pos_list, depth_list, s=1.5, color="gray", alpha=0.7)
ax2.invert_yaxis()
ax2.set_title("particle volume")

ax3 = ax[1,0]
im3 = ax3.contourf(ctdX, ctdY, tempZ, 50, cmap="rainbow")
fig.colorbar(im3, orientation='vertical', ax=ax3)
ax3.scatter(ctd_pos_list, ctd_depth_list, s=1.5, color="gray", alpha=0.7)
ax3.invert_yaxis()
ax3.set_title("temperature")

ax4 = ax[1,1]
im4 = ax4.contourf(ctdX, ctdY, salZ, 50, cmap="rainbow")
fig.colorbar(im4, orientation='vertical', ax=ax4)
ax4.scatter(ctd_pos_list, ctd_depth_list, s=1.5, color="gray", alpha=0.7)
ax4.invert_yaxis()
ax4.set_title("Salinity")

ax5 = ax[2,0]
im5 = ax5.contourf(ctdX, ctdY, oxyZ, 50, cmap="rainbow")
fig.colorbar(im5, orientation='vertical', ax=ax5)
ax5.scatter(ctd_pos_list, ctd_depth_list, s=1.5, color="gray", alpha=0.7)
ax5.invert_yaxis()
ax5.set_title("oxygen")

ax6 = ax[2,1]
im6 = ax6.contourf(ctdX, ctdY, fluZ, 50, cmap="rainbow")
fig.colorbar(im6, orientation='vertical', ax=ax6)
ax6.scatter(ctd_pos_list, ctd_depth_list, s=1.0, color="gray", alpha=0.7)
ax6.invert_yaxis()
ax6.set_title("fluoresence")

fig.tight_layout()

figname = "tran3"
figpath = os.path.expanduser(os.path.join(plot_path, figname) )
fig.savefig(figpath, dpi=300, facecolor="white")
plt.close()
